# NMA example code

This script demonstrates how to load in data remotely from our ECoG dataset: https://gui.dandiarchive.org/#/dandiset/000055.

To use, go to https://hub.dandiarchive.org/, log in with your Github account, and start a tiny server. Then, create a new jupyter notebook, copy the cells below into it, and run the notebook.

In [ ]:
%pip install natsort
%pip install dandi
%pip install git+https://github.com/catalystneuro/brunton-lab-to-nwb.git


In [ ]:
import pdb
import glob
import time
import natsort
import numpy as np

# Imports to load in data remotely
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient
from ndx_events import LabeledEvents, AnnotatedEventsTable, Events
from nwbwidgets.utils.timeseries import align_by_times

## Set parameters

In [ ]:
part_ind = 1
n_move_evs = 10
tlims = [-1.5, 1.5]


## First, select *n_move_evs* movement events across the different recording days

In [ ]:
# Get all file paths for 1 participant
with DandiAPIClient() as client:
    paths = []
    for file in client.get_dandiset("000055", "draft").get_assets_under_path(''):
        paths.append(file.path)
paths = natsort.natsorted(paths)
fids = [val for val in paths if 'sub-'+str(part_ind).zfill(2) in val]

# Get all movement events for all files
times_all, fid_lst = [], []
for i, fid in enumerate(fids):
    with DandiAPIClient() as client:
        asset = client.get_dandiset("000055", "draft").get_asset_by_path(fid)
        s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)

    io = NWBHDF5IO(s3_path, mode='r', load_namespaces=False, driver='ros3')
    nwb = io.read()

    events = nwb.processing["behavior"].data_interfaces["ReachEvents"]
    times_all.extend(events.timestamps[:].tolist())
    fid_lst.extend([fid]*len(events.timestamps[:]))

# Randomly select 100 movement events
n_tot_evs = len(times_all)
assert n_move_evs <= n_tot_evs
ev_inds = np.random.permutation(n_tot_evs)[:n_move_evs]
times_all = np.asarray(times_all)[ev_inds]
fid_lst = np.asarray(fid_lst)[ev_inds]


## Segment the neural data at each movement event (will take very long for more movement events)

In [ ]:
tstart = time.time()

fids_uni = np.unique(fid_lst).tolist()
for i, fid in enumerate(fids_uni):
    with DandiAPIClient() as client:
        asset = client.get_dandiset("000055", "draft").get_asset_by_path(fid)
        s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)

    io = NWBHDF5IO(s3_path, mode='r', load_namespaces=False, driver='ros3')
    nwb = io.read()
    
    times = times_all[fid_lst == fid]
    starts = times + tlims[0]
    stops = times + tlims[1]
    
    print(fid)
    ep_dat_tmp = align_by_times(nwb.acquisition['ElectricalSeries'], starts, stops)
    
    if i == 0:
        ep_dat = ep_dat_tmp.copy()
    else:
        ep_dat = np.vstack((ep_dat, ep_dat_tmp.copy()))
    
print(ep_dat.shape)
print('Run time: {0:.2f} sec'.format(time.time()-tstart))
